# 03 — LSTM Forecasting with TensorFlow/Keras

**Goal:** Build an **LSTM** to forecast 1‑step ahead values for the energy proxy series.

**You will learn:**
- Turning a univariate price series into supervised sequences.
- Defining and training a compact LSTM.
- Evaluating and visualizing predictions.

> On Apple Silicon, if `pip install tensorflow` fails, try `pip install tensorflow-macos`.


## 0) Setup & Imports

In [ ]:
# If needed, install (uncomment)
# %pip install yfinance pandas numpy matplotlib scikit-learn tensorflow

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error
from math import sqrt
import yfinance as yf

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import EarlyStopping

plt.rcParams['figure.figsize'] = (12, 5)
plt.rcParams['axes.grid'] = True

print("TensorFlow:", tf.__version__)


## 1) Download data

In [ ]:
TICKER = "XLE"
df = yf.download(TICKER, start="2015-01-01").reset_index()
df = df[["Date", "Close"]].dropna().reset_index(drop=True)
df.head()


## 2) Create supervised sequences

In [ ]:
WINDOW = 30
values = df["Close"].values.reshape(-1, 1)

scaler = MinMaxScaler()
scaled = scaler.fit_transform(values)

def make_sequences(series, window):
    X, y = [], []
    for i in range(len(series) - window):
        X.append(series[i:i+window])
        y.append(series[i+window])
    return np.array(X), np.array(y)

X, y = make_sequences(scaled, WINDOW)

split = int(len(X) * 0.8)
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

X_train.shape, X_test.shape, y_train.shape, y_test.shape


## 3) Define & train the LSTM

In [ ]:
model = Sequential([
    LSTM(64, input_shape=(X_train.shape[1], X_train.shape[2])),
    Dense(1)
])
model.compile(optimizer="adam", loss="mse")
model.summary()

es = EarlyStopping(monitor="val_loss", patience=10, restore_best_weights=True)

history = model.fit(
    X_train, y_train,
    validation_split=0.2,
    epochs=200,
    batch_size=32,
    callbacks=[es],
    verbose=1
)


## 4) Evaluate & visualize

In [ ]:
y_pred = model.predict(X_test)

y_test_inv = scaler.inverse_transform(y_test)
y_pred_inv = scaler.inverse_transform(y_pred)

rmse = sqrt(mean_squared_error(y_test_inv, y_pred_inv))
mae = mean_absolute_error(y_test_inv, y_pred_inv)
mape = np.mean(np.abs((y_test_inv - y_pred_inv) / y_test_inv)) * 100

print("LSTM | RMSE:", rmse, "MAE:", mae, "MAPE%:", mape)

plt.figure()
plt.plot(df["Date"].values[-len(y_test_inv):], y_test_inv, label="Actual")
plt.plot(df["Date"].values[-len(y_test_inv):], y_pred_inv, label="LSTM")
plt.title("LSTM Forecast — Actual vs Predicted (Test)")
plt.legend(); plt.show()


## 5) Next steps

- Try different `WINDOW` sizes and batch sizes.
- Add **exogenous features** (e.g., natural gas `NG=F`) as additional input channels.
- Perform **walk‑forward retraining** to simulate live operation.
- Compare LSTM vs ARIMA vs LightGBM on the same rolling split.
